In [46]:
import os
import numpy as np
import pandas as pd
from scipy import stats, interpolate
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
# Download risk-free rate data
data_folder = r'D:\Backup\fintech-assignments\HW4\data'
url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldAll'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find('table')
rf = pd.read_html(str(table))[1]
rf.to_csv(os.path.join(data_folder, 'Daily_Treasury_Yield_Curve_Rates.csv'), index = False)

In [5]:
rf.head()

,Date,1 mo,2 mo,3 mo,6 mo,1 yr,2 yr,3 yr,5 yr,7 yr,10 yr,20 yr,30 yr
0,01/02/90,NaN,NaN,7.83,7.89,7.81,7.87,7.90,7.87,7.98,7.94,NaN,8.00
1,01/03/90,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04
2,01/04/90,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
3,01/05/90,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
4,01/08/90,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09


In [74]:
# Interpolate for 1-week risk-free rate 
rf_4week = rf.at[len(rf['Date'])-1, '1 mo']/100
rf_8week = rf.at[len(rf['Date'])-1, '2 mo']/100
rfweek_int = interpolate.interp1d([4, 8], [rf_4week, rf_8week], fill_value = 'extrapolate')

# risk-free rate --> rf = r*t
rf_1week = rfweek_int(1).item()*1/52

In [3]:
# Load etf data
crawldata_folder = r'D:\Backup\fintech-assignments\hw1_crawl_data\etf_price_info'
ETF = pd.DataFrame()
for ETFtype in os.listdir(os.path.join(crawldata_folder, 'price_csv_dl')):
    for ETFname in os.listdir(os.path.join(crawldata_folder, 'price_csv_dl', ETFtype)):
        path = os.path.join(crawldata_folder, 'price_csv_dl', ETFtype, ETFname)
        df1 = pd.read_csv(path, usecols = ['Date', 'Adj Close'], parse_dates = ['Date'])
        df1['Type'] = ETFtype
        df1['ETF'] = ETFname[:-4]
        ETF = ETF.append(df1)
ETF = ETF[['Date', 'Type', 'ETF', 'Adj Close']]

In [63]:
# Requirement to get meaningful parameters
def requirement(df):
    req = df['ret_kurt'] > 3+5/3*df['ret_skew']**2
    return req
    
# Parameters of NIG distribution
def ASKSR_parameter(df, parname, rf = rf_1week):
    mean = df['ret_mean']
    var = df['ret_var']
    skew = df['ret_skew']
    kurt = df['ret_kurt']
    
    alpha = (3*np.sqrt(3*kurt-4*skew**2-9))/(var*(3*kurt-5*skew**2-9))
    beta = (3*skew)/(np.sqrt(var)*(3*kurt-5*skew**2-9))
    ita = mean-(3*skew*np.sqrt(var))/(3*kurt-4*skew**2-9)
    delta = 3*np.sqrt(var)*np.sqrt(3*kurt-5*skew**2-9)/(3*kurt-4*skew**2-9)
    phi = np.sqrt(alpha**2-beta**2)
    
    # a_ = (a*)*lambda  , where lambda is a measure of absolute risk aversion
    a_ = 1*(beta+alpha*(ita-rf)/np.sqrt(delta**2+(ita-rf)**2))
    
    ASKSR = np.sqrt(2*(a_*(ita-rf)-delta*(phi-np.sqrt(alpha**2-(beta-a_)**2))))
    
    if parname == 'alpha': return alpha
    elif parname == 'beta': return beta
    elif parname == 'ita': return ita
    elif parname == 'delta': return delta
    elif parname == 'phi': return phi
    elif parname == 'a*': return a_
    elif parname == 'ASKSR': return ASKSR
    else: return float('nan')

In [112]:
# Calculate weekly return
df = pd.DataFrame(pd.date_range(start = '20160106', end = '20190515', freq = '7D', name = 'Date'))
weekly_return = ETF[ETF['Date'].isin(df['Date'])]
weekly_return['wret'] = weekly_return.groupby(['ETF'])['Adj Close'].pct_change()
weekly_return.to_csv(os.path.join(data_folder, 'ETF_weeklyreturn.csv'), index = False)
weekly_return.head()

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Date,Type,ETF,Adj Close,wret
3,2016-01-06,ConsumerStaplesEquity,ECON,20.061361,NaN
8,2016-01-13,ConsumerStaplesEquity,ECON,18.964792,-0.054661
12,2016-01-20,ConsumerStaplesEquity,ECON,18.289227,-0.035622
17,2016-01-27,ConsumerStaplesEquity,ECON,18.935419,0.035332
22,2016-02-03,ConsumerStaplesEquity,ECON,19.806799,0.046019


In [113]:
# Calculate weekly ASKSR
weeklyASKSR = pd.DataFrame()

# Estimation of the moments: variance, sigma, skew, kurt
weeklyASKSR['ETF'] = weekly_return.groupby(['ETF']).groups.keys()
weeklyASKSR['ret_mean'] = list(weekly_return.groupby(['ETF'])['wret'].mean())
weeklyASKSR['ret_var'] = list(weekly_return.groupby(['ETF'])['wret'].var())
weeklyASKSR['ret_skew'] = list(weekly_return.groupby(['ETF'])['wret'].apply(lambda s: stats.skew(s.iloc[1:])))
weeklyASKSR['ret_kurt'] = list(weekly_return.groupby(['ETF'])['wret'].apply(lambda s: stats.kurtosis(s.iloc[1:])))

# Most of the ETF fail to meet the requirement, and thus would get strange parameters.
weeklyASKSR['reqirement'] = weeklyASKSR.apply(requirement, axis = 1)
weeklyASKSR = weeklyASKSR[weeklyASKSR['reqirement'] == True]

weeklyASKSR['alpha'] = weeklyASKSR.apply(parameter, axis = 1, args = ('alpha',))
weeklyASKSR['beta'] = weeklyASKSR.apply(parameter, axis = 1, args = ('beta',))
weeklyASKSR['ita'] = weeklyASKSR.apply(parameter, axis = 1, args = ('ita',))
weeklyASKSR['delta'] = weeklyASKSR.apply(parameter, axis = 1, args = ('delta',))
weeklyASKSR['phi'] = weeklyASKSR.apply(parameter, axis = 1, args = ('phi',))
weeklyASKSR['a*'] = weeklyASKSR.apply(parameter, axis = 1, args = ('a*',))
weeklyASKSR['ASKSR'] = weeklyASKSR.apply(parameter, axis = 1, args = ('ASKSR',))

# Rank the ETF by ASKSR
weeklyASKSR['wrank'] = weeklyASKSR['ASKSR'].rank()

weeklyASKSR.head()

,ETF,ret_mean,ret_var,ret_skew,ret_kurt,reqirement,alpha,beta,ita,delta,phi,a*,ASKSR,wrank
0,BDCL,0.003156,0.001280,-0.814254,5.214008,True,1407.181657,-20.522393,0.025060,0.049066,1407.031999,610.158838,NaN,NaN
1,BDCS,0.001679,0.000315,-0.855807,4.679683,True,10045.051203,-105.051111,0.023281,0.029620,10044.501876,6025.649354,NaN,NaN
2,BDCZ,0.001850,0.000260,0.172349,8.503940,True,2857.133120,1.960358,0.001342,0.011932,2857.132447,212.482895,NaN,NaN
3,BIZD,0.001988,0.000316,-0.709241,4.399561,True,8326.344600,-71.043352,0.019299,0.031669,8326.041511,4185.792101,NaN,NaN
35,LBDC,0.003178,0.001314,-0.396989,4.001422,True,1587.602517,-14.826396,0.021362,0.068190,1587.533285,450.443139,NaN,NaN


In [103]:
# Calculate monthly return
month_check = []
date_list = []
for date in sorted(ETF['Date'].dt.strftime('%Y%m%d').unique(), reverse = True):
    if not date[0:6] in month_check:
        month_check.append(date[0:6])
        date_list.append(date)

monthly_return = ETF[ETF['Date'].isin(date_list)]
monthly_return['mret'] = monthly_return.groupby(['ETF'])['Adj Close'].pct_change()
monthly_return.to_csv(os.path.join(data_folder, 'ETF_monthlyreturn.csv'), index = False)
monthly_return.head()

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Date,Type,ETF,Adj Close,wret
0,2015-12-31,ConsumerStaplesEquity,ECON,20.815252,NaN
19,2016-01-29,ConsumerStaplesEquity,ECON,20.110317,-0.033866
39,2016-02-29,ConsumerStaplesEquity,ECON,19.464121,-0.032133
61,2016-03-31,ConsumerStaplesEquity,ECON,22.127222,0.136821
82,2016-04-29,ConsumerStaplesEquity,ECON,22.323036,0.008849


In [ ]:
# Calculate monthly ASKSR
monthlyASKSR = pd.DataFrame()

# Estimation of the moments: variance, sigma, skew, kurt
monthlyASKSR['ETF'] = monthly_return.groupby(['ETF']).groups.keys()
monthlyASKSR['ret_mean'] = list(monthly_return.groupby(['ETF'])['mret'].mean())
monthlyASKSR['ret_var'] = list(monthly_return.groupby(['ETF'])['mret'].var())
monthlyASKSR['ret_skew'] = list(monthly_return.groupby(['ETF'])['mret'].apply(lambda s: stats.skew(s.iloc[1:])))
monthlyASKSR['ret_kurt'] = list(monthly_return.groupby(['ETF'])['mret'].apply(lambda s: stats.kurtosis(s.iloc[1:])))

# Most of the ETF fail to meet the requirement, and thus would get strange parameters.
monthlyASKSR['reqirement'] = monthlyASKSR.apply(requirement, axis = 1)
monthlyASKSR = monthlyASKSR[monthlyASKSR['reqirement'] == True]

monthlyASKSR['alpha'] = monthlyASKSR.apply(parameter, axis = 1, args = ('alpha',))
monthlyASKSR['beta'] = monthlyASKSR.apply(parameter, axis = 1, args = ('beta',))
monthlyASKSR['ita'] = monthlyASKSR.apply(parameter, axis = 1, args = ('ita',))
monthlyASKSR['delta'] = monthlyASKSR.apply(parameter, axis = 1, args = ('delta',))
monthlyASKSR['phi'] = monthlyASKSR.apply(parameter, axis = 1, args = ('phi',))
monthlyASKSR['a*'] = monthlyASKSR.apply(parameter, axis = 1, args = ('a*',))
monthlyASKSR['ASKSR'] = monthlyASKSR.apply(parameter, axis = 1, args = ('ASKSR',))

# Rank the ETF by ASKSR
monthlyASKSR['mrank'] = monthlyASKSR['ASKSR'].rank()

In [108]:
monthlyASKSR['ETF'] = monthly_return.groupby(['ETF']).groups.keys()
monthlyASKSR['ret_mean'] = list(monthly_return.groupby(['ETF'])['mret'].mean())
monthlyASKSR['ret_var'] = list(monthly_return.groupby(['ETF'])['mret'].var())
monthlyASKSR['ret_skew'] = list(monthly_return.groupby(['ETF'])['mret'].apply(lambda s: stats.skew(s.iloc[1:])))
monthlyASKSR['ret_kurt'] = list(monthly_return.groupby(['ETF'])['mret'].apply(lambda s: stats.kurtosis(s.iloc[1:])))
monthlyASKSR.head()

,ETF,ret_mean,ret_var,ret_skew,ret_kurt,reqirement
0,BDCL,0.014323,0.006167,0.198485,1.313497,NaN
1,BDCS,0.007532,0.001540,0.120526,1.284017,NaN
2,BDCZ,0.007577,0.001585,0.473163,1.579354,NaN
3,BIZD,0.009118,0.001765,0.150825,1.474066,NaN
4,CHIX,0.007965,0.003850,-0.258890,-0.040788,NaN


In [ ]:
weekly_rank = weeklyASKSR[['ETF', 'wrank']].sort_values(by = ['wrank'])
weekly_rank.to_csv(os.path.join(data_folder, 'ETF_monthlyreturn.csv'), index = False)
monthly_rank = monthlyASKSR[['ETF', 'mrank']].sort_values(by = ['mrank'])

In [74]:
# In the presence of short sale constraint
# if a* < 0, ASKSR = 0

# mean = ita+delta*beta/phi
# variance = delta*aplha**2/phi**3
# skew = 3*beta/(alpha*np.sqrt(delta*phi))
# kurt = 3+3/(delta*phi)*(1+4*(beta/alpha)**2)